In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # maximum number of tokens that the model can handle in a single input.
dtype = None # None for data type auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no Out Of Memory issues (OOMs).
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth


#Loading the Model 
model, tokenizer = FastLanguageModel.from_pretrained(
    #Loads a pre-trained model along with its tokenizer.
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

Add LoRA adapters so we only need to update 1 to 10% of all parameters! This techniques allow us to fine-tune large models efficiently by adjusting only a subset of parameters instead of the entire model, reducing computational cost and memory usage.

In [ ]:

# Use PEFT (Parameter-Efficient Fine-Tunning) on our selected model 
model = FastLanguageModel.get_peft_model(
    model,
    # rank of the LoRA matrix. The rank determines the number of parameters added for adaptation.
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128

    # which parts (modules) of the model to apply LoRA fine-tuning.
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],

    lora_alpha = 16, #how much influence the fine-tuned weights have on the original model.

    #Apply dropout to LoRA layers during fine-tuning to prevent overfitting
    lora_dropout = 0, # Supports any, but = 0 is optimized

    bias = "none",    # Supports any, but = "none" is optimized

    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context

    random_state = 3407, #seed for reproducibility, ensuring that results remain consistent across runs.

    # Rank-Stabilized LoRA (RS-LoRA), which helps stabilize training when fine-tuning models with very high rank 
    use_rslora = False,  #  disable support rank stabilized LoRA (default)

    loftq_config = None, # And LoftQ
)

In [ ]:
from unsloth.chat_templates import get_chat_template
from datasets import load_dataset

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.2",
)

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    
    texts = [
        tokenizer.apply_chat_template(
            {
                "instruction": instruction,
                "input": input_data,
                "output": output,
            },
            tokenize=False,
            add_generation_prompt=False,
        )
        for instruction, input_data, output in zip(instructions, inputs, outputs)
    ]
    return {"text": texts}

dataset = load_dataset("json", data_files="/Users/jabinwade/Coding/CareConnect/training_data/Diseases_Symptoms_training.jsonl", split="train")

In [ ]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
dataset[5] ["conversations"]

In [ ]:
dataset[5]["text"]